# Day 16: Introduction to Options

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/astoreyai/money-talks/blob/main/class3_trading_strategies/week4_options_basics/day16_intro_options.ipynb)

---

## Learning Objectives

1. **Understand** what options are and how they work
2. **Learn** key options terminology
3. **Identify** the components of an options contract
4. **Distinguish** between calls and puts
5. **Recognize** when options might be useful

---

## Lecture (30 minutes)

### What is an Option?

An **option** is a contract that gives the buyer the right, but not the obligation, to buy or sell an underlying asset at a specified price before a certain date.

```
OPTIONS BASICS
==============

DEFINITION:
A contract between two parties:
- BUYER: Pays premium, gains right
- SELLER: Receives premium, has obligation

TWO TYPES:
CALL OPTION: Right to BUY at strike price
PUT OPTION:  Right to SELL at strike price

REAL WORLD ANALOGY:
Option = Insurance or Deposit

Home deposit example:
- You pay $10,000 deposit (premium)
- Lock in $300,000 price (strike price)
- Valid for 3 months (expiration)
- You can walk away, lose deposit
- Seller must sell if you want to buy
```

### Key Options Terminology

```
OPTIONS VOCABULARY
==================

STRIKE PRICE (Exercise Price):
- Price at which option can be exercised
- Fixed when contract is created
- Example: $150 strike on AAPL

PREMIUM:
- Price paid for the option contract
- Paid by buyer to seller
- Example: $5.00 per share ($500 per contract)

EXPIRATION DATE:
- Last day option can be exercised
- Options expire worthless after this
- Weekly, monthly, quarterly, LEAPS (1-2 years)

UNDERLYING:
- The asset the option is based on
- Usually 100 shares per contract
- Can be stocks, ETFs, indexes

CONTRACT SIZE:
- Standard: 1 contract = 100 shares
- Premium of $5 = $500 total cost
```

### Calls vs Puts

```
CALL OPTIONS
============

CALL = Right to BUY

BUY CALL when you're BULLISH:
- Pay premium
- Profit if stock rises above strike + premium
- Max loss = premium paid
- Max gain = unlimited

SELL CALL when you're NEUTRAL/BEARISH:
- Receive premium
- Keep premium if stock stays below strike
- Max gain = premium received
- Max loss = unlimited (unless covered)

PUT OPTIONS
===========

PUT = Right to SELL

BUY PUT when you're BEARISH:
- Pay premium
- Profit if stock falls below strike - premium
- Max loss = premium paid
- Max gain = strike price - premium (stock to $0)

SELL PUT when you're BULLISH/NEUTRAL:
- Receive premium
- Keep premium if stock stays above strike
- Max gain = premium received
- Max loss = strike - premium (stock to $0)
```

### Moneyness

```
OPTION MONEYNESS
================

                CALL              PUT
Stock > Strike  IN THE MONEY     OUT OF THE MONEY
Stock = Strike  AT THE MONEY     AT THE MONEY
Stock < Strike  OUT OF THE MONEY IN THE MONEY

EXAMPLE (AAPL at $175):

$170 Call = IN THE MONEY (ITM)
  - Has $5 intrinsic value
  - Would profit if exercised now

$175 Call = AT THE MONEY (ATM)
  - No intrinsic value
  - All time value

$180 Call = OUT OF THE MONEY (OTM)
  - No intrinsic value
  - All time value (speculative)

ITM options cost more but are safer
OTM options are cheaper but riskier
```

### Options Value Components

```
OPTION PREMIUM COMPONENTS
=========================

Premium = Intrinsic Value + Time Value

INTRINSIC VALUE:
- "Real" value if exercised now
- Call: Stock Price - Strike (if positive)
- Put: Strike - Stock Price (if positive)
- Minimum value = $0

TIME VALUE (Extrinsic Value):
- Premium above intrinsic value
- Reflects possibility of future movement
- Decays as expiration approaches
- Higher with more time, more volatility

EXAMPLE:
AAPL at $175
$170 Call trading at $8.00

Intrinsic = $175 - $170 = $5.00
Time Value = $8.00 - $5.00 = $3.00
```

### Why Use Options?

```
OPTIONS USE CASES
=================

1. LEVERAGE
   - Control 100 shares for fraction of cost
   - Amplified returns (and losses)

2. HEDGING (Insurance)
   - Protect stock positions
   - Buy puts on stocks you own

3. INCOME GENERATION
   - Sell options on stocks you own
   - Covered calls, cash-secured puts

4. SPECULATION
   - Bet on direction with limited risk
   - Defined max loss = premium

5. STRATEGIC FLEXIBILITY
   - Profit from any direction or range
   - Complex strategies for any outlook

RISKS:
- Can lose 100% of premium
- Time decay works against buyers
- Leverage magnifies losses too
- Complexity can lead to mistakes
```

---

## Hands-On Practice (15 minutes)

In [ ]:
!pip install yfinance pandas numpy matplotlib -q
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
print("Libraries loaded!")

In [ ]:
def get_options_chain(ticker):
    """Get options chain for a stock."""
    stock = yf.Ticker(ticker)
    
    # Get expiration dates
    expirations = stock.options
    
    # Get current price
    info = stock.info
    current_price = info.get('currentPrice', info.get('regularMarketPrice', 0))
    
    print(f"\n{'='*50}")
    print(f"OPTIONS OVERVIEW: {ticker}")
    print(f"{'='*50}")
    print(f"\nCurrent Stock Price: ${current_price:.2f}")
    print(f"\nAvailable Expirations ({len(expirations)} dates):")
    for i, exp in enumerate(expirations[:5]):
        print(f"  {i+1}. {exp}")
    if len(expirations) > 5:
        print(f"  ... and {len(expirations)-5} more")
    
    return stock, current_price, expirations

ticker = 'AAPL'
stock, current_price, expirations = get_options_chain(ticker)

In [ ]:
def analyze_option_chain(stock, expiration, current_price):
    """Analyze calls and puts for a specific expiration."""
    options = stock.option_chain(expiration)
    calls = options.calls
    puts = options.puts
    
    print(f"\n{'='*60}")
    print(f"OPTIONS CHAIN: Expiration {expiration}")
    print(f"{'='*60}")
    
    # Filter to near-the-money options
    atm_range = current_price * 0.10  # 10% range
    
    calls_filtered = calls[
        (calls['strike'] >= current_price - atm_range) &
        (calls['strike'] <= current_price + atm_range)
    ].copy()
    
    puts_filtered = puts[
        (puts['strike'] >= current_price - atm_range) &
        (puts['strike'] <= current_price + atm_range)
    ].copy()
    
    # Calculate moneyness
    def moneyness(row, option_type):
        if option_type == 'call':
            if row['strike'] < current_price:
                return 'ITM'
            elif row['strike'] > current_price:
                return 'OTM'
            else:
                return 'ATM'
        else:  # put
            if row['strike'] > current_price:
                return 'ITM'
            elif row['strike'] < current_price:
                return 'OTM'
            else:
                return 'ATM'
    
    calls_filtered['Moneyness'] = calls_filtered.apply(lambda x: moneyness(x, 'call'), axis=1)
    puts_filtered['Moneyness'] = puts_filtered.apply(lambda x: moneyness(x, 'put'), axis=1)
    
    # Display calls
    print(f"\nCALL OPTIONS (Right to BUY at strike):")
    print(f"{'-'*60}")
    call_display = calls_filtered[['strike', 'lastPrice', 'bid', 'ask', 'volume', 'openInterest', 'Moneyness']].copy()
    call_display.columns = ['Strike', 'Last', 'Bid', 'Ask', 'Volume', 'OI', 'Status']
    print(call_display.round(2).to_string(index=False))
    
    # Display puts
    print(f"\nPUT OPTIONS (Right to SELL at strike):")
    print(f"{'-'*60}")
    put_display = puts_filtered[['strike', 'lastPrice', 'bid', 'ask', 'volume', 'openInterest', 'Moneyness']].copy()
    put_display.columns = ['Strike', 'Last', 'Bid', 'Ask', 'Volume', 'OI', 'Status']
    print(put_display.round(2).to_string(index=False))
    
    return calls_filtered, puts_filtered

# Analyze nearest expiration
if len(expirations) > 0:
    calls, puts = analyze_option_chain(stock, expirations[0], current_price)

In [ ]:
def calculate_option_value(stock_price, strike, premium, option_type='call'):
    """Calculate intrinsic and time value of an option."""
    if option_type == 'call':
        intrinsic = max(0, stock_price - strike)
    else:  # put
        intrinsic = max(0, strike - stock_price)
    
    time_value = premium - intrinsic
    
    return intrinsic, time_value

def analyze_option_components(calls, puts, current_price):
    """Analyze intrinsic and time value components."""
    print(f"\n{'='*60}")
    print("OPTION VALUE BREAKDOWN")
    print(f"{'='*60}")
    print(f"\nStock Price: ${current_price:.2f}")
    
    print(f"\nCALL OPTIONS:")
    print(f"{'-'*60}")
    print(f"{'Strike':<10}{'Premium':<10}{'Intrinsic':<12}{'Time Value':<12}{'Status':<10}")
    print(f"{'-'*60}")
    
    for _, row in calls.head(6).iterrows():
        intrinsic, time_val = calculate_option_value(
            current_price, row['strike'], row['lastPrice'], 'call'
        )
        print(f"${row['strike']:<9.2f}${row['lastPrice']:<9.2f}${intrinsic:<11.2f}${time_val:<11.2f}{row['Moneyness']:<10}")
    
    print(f"\nPUT OPTIONS:")
    print(f"{'-'*60}")
    print(f"{'Strike':<10}{'Premium':<10}{'Intrinsic':<12}{'Time Value':<12}{'Status':<10}")
    print(f"{'-'*60}")
    
    for _, row in puts.head(6).iterrows():
        intrinsic, time_val = calculate_option_value(
            current_price, row['strike'], row['lastPrice'], 'put'
        )
        print(f"${row['strike']:<9.2f}${row['lastPrice']:<9.2f}${intrinsic:<11.2f}${time_val:<11.2f}{row['Moneyness']:<10}")

if 'calls' in dir() and 'puts' in dir():
    analyze_option_components(calls, puts, current_price)

In [ ]:
def plot_option_payoff(strike, premium, option_type='call', position='long'):
    """Plot option payoff diagram."""
    # Range of stock prices
    stock_range = np.linspace(strike * 0.7, strike * 1.3, 100)
    
    # Calculate payoff
    if option_type == 'call':
        if position == 'long':
            payoff = np.maximum(stock_range - strike, 0) - premium
            title = f'Long Call: Strike ${strike}, Premium ${premium}'
            color = 'green'
        else:  # short
            payoff = premium - np.maximum(stock_range - strike, 0)
            title = f'Short Call: Strike ${strike}, Premium ${premium}'
            color = 'red'
    else:  # put
        if position == 'long':
            payoff = np.maximum(strike - stock_range, 0) - premium
            title = f'Long Put: Strike ${strike}, Premium ${premium}'
            color = 'blue'
        else:  # short
            payoff = premium - np.maximum(strike - stock_range, 0)
            title = f'Short Put: Strike ${strike}, Premium ${premium}'
            color = 'orange'
    
    # Plot
    plt.figure(figsize=(10, 6))
    plt.plot(stock_range, payoff, color=color, lw=2)
    plt.axhline(y=0, color='black', linestyle='-', lw=0.5)
    plt.axvline(x=strike, color='gray', linestyle='--', label=f'Strike: ${strike}')
    
    # Fill profit/loss areas
    plt.fill_between(stock_range, payoff, 0, where=payoff > 0, alpha=0.3, color='green', label='Profit')
    plt.fill_between(stock_range, payoff, 0, where=payoff < 0, alpha=0.3, color='red', label='Loss')
    
    plt.xlabel('Stock Price at Expiration')
    plt.ylabel('Profit/Loss per Share')
    plt.title(title)
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    # Add annotations
    if position == 'long':
        plt.annotate(f'Max Loss: ${premium:.2f}', xy=(strike * 0.75, -premium), fontsize=10)
        if option_type == 'call':
            be = strike + premium
            plt.annotate(f'Breakeven: ${be:.2f}', xy=(be, 0.5), fontsize=10)
        else:
            be = strike - premium
            plt.annotate(f'Breakeven: ${be:.2f}', xy=(be, 0.5), fontsize=10)
    
    plt.tight_layout()
    plt.show()

# Example payoff diagrams
print("Long Call Option Payoff:")
plot_option_payoff(strike=175, premium=5, option_type='call', position='long')

print("\nLong Put Option Payoff:")
plot_option_payoff(strike=175, premium=4, option_type='put', position='long')

In [ ]:
def options_calculator(stock_price, strike, premium, option_type='call', contracts=1):
    """Interactive options calculator."""
    shares = contracts * 100
    total_cost = premium * shares
    
    print(f"\n{'='*50}")
    print("OPTIONS TRADE CALCULATOR")
    print(f"{'='*50}")
    print(f"\nOption Type: {option_type.upper()}")
    print(f"Strike Price: ${strike}")
    print(f"Premium: ${premium} per share")
    print(f"Contracts: {contracts} ({shares} shares)")
    print(f"Total Cost: ${total_cost:,.2f}")
    
    # Breakeven
    if option_type == 'call':
        breakeven = strike + premium
    else:
        breakeven = strike - premium
    
    print(f"\nBreakeven Price: ${breakeven:.2f}")
    
    # Max loss
    max_loss = total_cost
    print(f"Max Loss (if expires worthless): ${max_loss:,.2f}")
    
    # Scenarios
    print(f"\n{'-'*50}")
    print("PROFIT/LOSS SCENARIOS:")
    print(f"{'-'*50}")
    
    scenarios = [strike * 0.9, strike * 0.95, strike, strike * 1.05, strike * 1.1]
    
    for price in scenarios:
        if option_type == 'call':
            profit = (max(0, price - strike) - premium) * shares
        else:
            profit = (max(0, strike - price) - premium) * shares
        
        pct_return = (profit / total_cost) * 100
        print(f"  Stock at ${price:.2f}: P/L = ${profit:+,.2f} ({pct_return:+.1f}%)")

# Example calculation
options_calculator(stock_price=175, strike=180, premium=3.50, option_type='call', contracts=2)

---

## Quiz

In [ ]:
quiz = [
    {"q": "A call option gives the buyer the right to?", "a": "A", 
     "opts": ["A) Buy the stock at the strike price", "B) Sell the stock at the strike price", 
              "C) Buy the stock at market price", "D) Sell the stock at market price"]},
    {"q": "One standard options contract controls how many shares?", "a": "C",
     "opts": ["A) 10 shares", "B) 50 shares", 
              "C) 100 shares", "D) 1000 shares"]},
    {"q": "A call option is IN THE MONEY when?", "a": "B",
     "opts": ["A) Stock price < Strike price", "B) Stock price > Strike price", 
              "C) Stock price = Strike price", "D) Premium > Strike price"]},
    {"q": "The maximum loss for buying a call option is?", "a": "A",
     "opts": ["A) The premium paid", "B) The strike price", 
              "C) Unlimited", "D) The stock price"]},
    {"q": "Time value in an option represents?", "a": "D",
     "opts": ["A) Current profit if exercised", "B) The stock's dividend", 
              "C) The strike price", "D) Value from time remaining until expiration"]}
]

def run_quiz():
    for i, q in enumerate(quiz):
        print(f"\nQ{i+1}: {q['q']}")
        for opt in q['opts']:
            print(f"   {opt}")
    print("\n" + "="*50)
    print("ANSWERS: 1-A, 2-C, 3-B, 4-A, 5-D")
    
run_quiz()

---

## Summary

- **Options** give the right, not obligation, to buy or sell
- **Calls** = right to buy, **Puts** = right to sell
- **Premium** = Intrinsic Value + Time Value
- **ITM/ATM/OTM** describes relationship to current price
- **1 contract** = 100 shares

**Tomorrow**: Day 17 - Call Options Deep Dive